# Project Setup Verification

This notebook verifies that the environment is set up correctly, including:
1. **Dependencies**: Checks if required libraries are installed.
2. **Data Paths**: Confirms that data files exist in `data/processed`.
3. **LLM Fallback**: Tests the robustness of the LLM configuration.

In [1]:
import sys
import os
from pathlib import Path

# Add src to path
project_root = Path("..").resolve()
sys.path.append(str(project_root / "src"))

print(f"Project Root: {project_root}")

Project Root: C:\Development\financial-intelligence-engine


## 1. Check Dependencies

In [2]:
try:
    import langchain
    import langchain_core
    import langchain_openai
    from services.llm_services import get_llm
    print("SUCCESS: Critical dependencies imported.")
except ImportError as e:
    print(f"FAILURE: Missing dependency: {e}")

SUCCESS: Critical dependencies imported.


## 2. Check Data Paths

In [3]:
processed_dir = project_root / "data/processed"
files_to_check = ["train.jsonl", "golden_test_set.jsonl", "qa_dataset_full.json"]

all_exist = True
for filename in files_to_check:
    path = processed_dir / filename
    if path.exists():
        print(f"SUCCESS: Found {filename}")
    else:
        print(f"FAILURE: Missing {filename} at {path}")
        all_exist = False

if all_exist:
    print("\nAll data files present in 'data/processed'.")
else:
    print("\nSome data files are missing. Please run '01_data_factory.ipynb' first.")

SUCCESS: Found train.jsonl
SUCCESS: Found golden_test_set.jsonl
SUCCESS: Found qa_dataset_full.json

All data files present in 'data/processed'.


## 3. Verify LLM Fallback Logic

In [4]:
from langchain_core.runnables import RunnableWithFallbacks

print("Testing Fallback Configuration Construction...")

# Mock Config with Multi-Provider Fallback
config = {
    "llm_provider": "openai",
    "llm_model": "gpt-4o",
    "openai_api_key": "sk-fake-key", # Fake key, we only test construction
    "enable_fallback": True,
    "fallbacks": [
        {"llm_provider": "openai", "llm_model": "gpt-3.5-turbo"},
        {"llm_provider": "openai", "llm_model": "gpt-4-turbo"}
    ]
}

try:
    # force a simplistic creation to avoid actual API calls if possible,
    # but get_llm might try to validate. 
    # If it fails due to API key, we catch it.
    # However, langchain usually lazy-initializes, so construction might pass.
    llm = get_llm(config)
    
    if isinstance(llm, RunnableWithFallbacks):
        print("SUCCESS: LLM instance is correctly wrapped with RunnableWithFallbacks")
        print(f"Primary Model: {llm.runnable.model_name if hasattr(llm.runnable, 'model_name') else llm.runnable}")
        print(f"Fallback Models Count: {len(llm.fallbacks)}")
        if len(llm.fallbacks) == 2:
             print("SUCCESS: Correct number of fallbacks (2) instantiated.")
        else:
             print(f"FAILURE: Expected 2 fallbacks, got {len(llm.fallbacks)}")
    else:
        print("FAILURE: LLM instance is NOT wrapped with RunnableWithFallbacks")

except Exception as e:
    print(f"Check: {e}")

Testing Fallback Configuration Construction...
Check: 'temperature'
